# <font color='red'>Question 1</font>

# <font color='green'>Working with AIF360 on IBM Watson
    
    - Bias and Fairness analysis on loan defaulters prediction

In [1]:
#Firstly install Aif360 package -> Uncomment the below line
!pip install aif360
!pip install cvxpy==0.4.11

     |████████████████████████████████| 56.4MB 55.1MB/s eta 0:00:01
     |████████████████████████████████| 20.4MB 37.0MB/s eta 0:00:01
ERROR: tensorflow 1.13.1 requires tensorboard<1.14.0,>=1.13.0, which is not installed.
  Found existing installation: numpy 1.15.4
    Uninstalling numpy-1.15.4:
      Successfully uninstalled numpy-1.15.4
     |████████████████████████████████| 163kB 16.3MB/s eta 0:00:01
     |████████████████████████████████| 153kB 22.5MB/s eta 0:00:01
     |████████████████████████████████| 163kB 30.1MB/s eta 0:00:01
     |████████████████████████████████| 1.6MB 30.0MB/s eta 0:00:01
     |████████████████████████████████| 696kB 44.2MB/s eta 0:00:01
     |████████████████████████████████| 153kB 46.4MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/2d/26/ba/f0a0388a7ab98124caf88ea8b532b81cf4acb1b6a4a990e3b4
  Stored in directory: /home/dsxuser/.cache/pip/wheels/68/3f/24/e9c75d426f600634cdac68321184ba06fdc4ab2d189b5c4541
  Stored in directory: /ho

In [2]:
# Load all necessary packages

import sys
sys.path.insert(1, "../")  #this will append a new location to accept environment variables from

import os

import numpy as np
np.random.seed(0)

from aif360.datasets import GermanDataset #Data set

from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing

from IPython.display import Markdown, display

In [3]:
#Adding a location to read data from

aif360_location = !python -c "from distutils.sysconfig import get_python_lib; print(get_python_lib())"

install_loc = os.path.join(aif360_location[0], "aif360/data/raw/german/")

#Checking the current directory
%cd $install_loc

/opt/conda/envs/Python36/lib/python3.6/site-packages/aif360/data/raw/german


In [4]:
#Loading the actual dataset to local location from module ftp
!wget ftp://ftp.ics.uci.edu/pub/machine-learning-databases/statlog/german/german.data
!wget ftp://ftp.ics.uci.edu/pub/machine-learning-databases/statlog/german/german.doc

--2019-08-21 11:12:26--  ftp://ftp.ics.uci.edu/pub/machine-learning-databases/statlog/german/german.data
           => ‘german.data.1’
Resolving ftp.ics.uci.edu (ftp.ics.uci.edu)... 128.195.1.14
Connecting to ftp.ics.uci.edu (ftp.ics.uci.edu)|128.195.1.14|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/machine-learning-databases/statlog/german ... done.
==> SIZE german.data ... 79793
==> PASV ... done.    ==> RETR german.data ... done.
Length: 79793 (78K) (unauthoritative)

100%[======================================>] 79,793      --.-K/s   in 0.07s   

2019-08-21 11:12:27 (1.02 MB/s) - ‘german.data.1’ saved [79793]

--2019-08-21 11:12:28--  ftp://ftp.ics.uci.edu/pub/machine-learning-databases/statlog/german/german.doc
           => ‘german.doc.1’
Resolving ftp.ics.uci.edu (ftp.ics.uci.edu)... 128.195.1.14
Connecting to ftp.ics.uci.edu (ftp.ics.uci.edu)|128.195.1.14|:21... connected.
Logging in as a

In [5]:
%cd -
dataset_orig = GermanDataset(protected_attribute_names=['age'],
# this dataset also contains protected
# attribute for "sex" which we do not
# consider in this evaluation
privileged_classes=[lambda x: x >= 25],      # age >=25 is considered privileged
features_to_drop=['personal_status', 'sex']) # ignore sex-related attributes

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

/home/dsxuser/work


In [6]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.169905


In [7]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(dataset_orig_train)

In [8]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.000000


In [9]:
file=open("fairness.txt", 'a+')
file.write("Bias = 0.0")
file.close

<function TextIOWrapper.close()>

# <font color='green'>Summary</font>

- The average difference between privilidged and under privileged groups is 0, this basically means there is no bias given the selection of our metric.
- There can be several metrics to check bias and fairness of a machine learning model based on the domain of work
- It is recommended to always check as many times as possible for bias in the model and constantly work to correecting it, in order to create the best model for realtime usage.

# <font color='red'>Question 2</font>
# Keras exercise
- Creating a Keras model by loading an inbuilt data set
- preprocessing input data
- building a Sequential Keras model and compiling the model with a training configuration
- model evaluation

# Goal
- Create a Multi-layer perceptron (MLP) using Keras which is trained to classify news items into the specified 46 topics.



In [10]:
#Installing Keras if not installed already
try:
    __import__('keras')
except ImportError:
    pip.main(['install', 'keras']) 
    
    
#Importing required functions
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical

Using TensorFlow backend.
/opt/conda/envs/Python36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/envs/Python36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/envs/Python36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/envs/Python36/lib/python3.6/site-packag

# Data
- Reuters newswire dataset.
         *This dataset consists of 11,228 newswires from the Reuters news agency. Each wire is encoded as a sequence of word indexes. Moreover, each wire is categorised into one of 46 topics, which will serve as our label. This dataset is available through the Keras API.

In [11]:
#This step is to change the default loading parameters of Numpy module
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

In [12]:
from keras.datasets import reuters #importing the dataset

#Train test validation split in 80/20 ratio
max_words = 1000
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2,
                                                         seed=1344)
num_classes = np.max(y_train) + 1 #number of categories will be the max_val of y_train+1 because the indexing would've started from 0

2113536/2110848 [==============================] - 0s 0us/step


In [13]:
# restore np.load to its default parameters
np.load = np_load_old

In [14]:
#tokenizing words is to split each word separately, whihc gives us each instance in the data as a separate attribute

from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')

In [15]:
y_train = to_categorical(y_train, num_classes=num_classes) #changing the taget variable to categories
y_test = to_categorical(y_test, num_classes=num_classes)

In [16]:
#defining a ANN model

model = Sequential()  # Instantiate sequential model

model.add(Dense(512, activation='relu', input_shape = (max_words,))) # Add first layer. Make sure to specify input shape

model.add(Dropout(0.5)) # Add second layer with dropout

model.add(Dense(num_classes, activation='softmax')) # Add third layer with softmax because there are more than 2 classes

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [17]:
#compiling the model with optimizer, loss and mrelevant metrics
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

In [ ]:
#Fitting the model with dezired parameters

batch_size = 32
model.fit(x_train, y_train, batch_size=batch_size, epochs=5, validation_data=(x_test,y_test))
score = model.evaluate(x_test,y_test, verbose=1)

Instructions for updating:
Use tf.cast instead.
Train on 8982 samples, validate on 2246 samples
Epoch 1/5
 192/8982 [..............................] - ETA: 1:24 - loss: 3.5677 - acc: 0.1875 

/opt/conda/envs/Python36/lib/python3.6/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.348934). Check your callbacks.
  % delta_t_median)


8982/8982 [==============================] - 24s 3ms/step - loss: 1.4054 - acc: 0.6860 - val_loss: 0.9260 - val_acc: 0.7934
Epoch 2/5
8982/8982 [==============================] - 23s 3ms/step - loss: 0.7913 - acc: 0.8126 - val_loss: 0.7585 - val_acc: 0.8281
Epoch 3/5
1216/8982 [===>..........................] - ETA: 22s - loss: 0.5156 - acc: 0.8849

In [ ]:
print("Loss: ",score[0],"\nAccuracy:",score[1])  #From the above extracting only the final and best response during 5th epoch

In [ ]:
file=open("loss_acc.txt", 'a+')
file.write("Loss = 73.72 + \n +Accuracy= 82.01")
file.close

# <font color='green'>Summary

- The MLP model created using Keras API on Watson has successfully classified the newswires into the designated 48 categories with 82.01 accuracy.